# Introduction

This notebook presents the several machine learning models using CNN and LSTM for HAR. To obtain a detailed description of the architecture, please refer to the dissertation, **"RECOGNISING HUMAN ACTIVITIES AUTONOMOUSLY THROUGH FUSION OF SENSOR DATA"**.

## Dataset

As a dataset, the [OPPORTUNITY Activity Recognition Data Set](http://archive.ics.uci.edu/ml/datasets/OPPORTUNITY+Activity+Recognition) is used. To prepare this dataset for the program, the following code is uncommented and executed.


In [ ]:
# Download dataset zip file and place data files in training and test set directries

zipfile_dataset_opportunity = "OpportunityUCIDataset.zip"
url_dataset_opportunity = "https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip"

#!wget $url_dataset_opportunity
#!unzip $zipfile_dataset_opportunity
#!ls OpportunityUCIDataset/dataset/

In [ ]:
# Deploy dataset files into training and test directories

#!mkdir -p ../data/test
#!mkdir ../data/train

#%cd OpportunityUCIDataset/dataset/
#!cp S[1-3]-Drill.dat S1-ADL[1-5].dat S2-ADL[1-3].dat S3-ADL[1-3].dat ../../../data/train/
#!cp S[23]-ADL[23].dat ../../../data/test/
#%cd ../../
#!ls ../data/train/
#!ls ../data/test/

# 1.Parameters

Adjustable flags and parameters are listed. Hyperparameters for each ML model are in "[F] ML models" section.

|Name|Type|Explanation|
|-|-|-|
|flag_delete_null|Flag|Whether delete the Null class or not|
|flag_label|Flag|Activity type (gesture or locomotion)|
|flag_(ML model name)|Flag|Whether execute the model or not|
|flag_experiment|Flag|Whether run repeated evaluation for summary statistics or not|
|flag_model_load|Flag|Whether load the model from the file or not|
|flag_model_save|Flag|Whether save the model to the file after training or not|
|flag_EarlyStopping|Flag|Enable Early stopping|
|flag_es_monitor|Flag|Monitor type for Early stopping|
|ratio_train|Parameter|The ratio between training and validation sets|
|seed|Parameter|Fix the seed for reproducibility|
|flag_scale|Flag|Scaling technique|
|window_size|Parameter|The length of the sliding window|
|window_step|Parameter|The step of the sliding window|
|flag_sw_label|Flag|Class label of the sliding window|
|flag_balance_*|Flag|Enable data balancing|
|flag_data_dist|Flag|Display dataset distribution|
|flag_interpolate|Flag|Enable interpolation|
|flag_plot_model|Flag|Whether plot model graphs or not|
|flag_save_fig|Flag|Whether save graphs or not|
|flag_summary|Flag|Show summary of the dataset|
|flag_cm_norm|Flag|Whether normalise confusion matrix or not|
|flag_TensorBoard|Flag|Whether save the Tensorboard data or not|

In [ ]:
### [Note]
#
# Hyperparameters for each ML model are in "[F] ML models" section
#
### ---------- ---------- ---------- ---------- ----------
### Flags
flag_delete_null    = False

# Label
flag_label          = "ML_Both_Arms"
#flag_label          = "Locomotion"

# ML
flag_CNN_1d         = True
flag_LSTM_Mto1      = True
flag_CNN1D_LSTM     = True
flag_ConvLSTM       = True
flag_Ensemble       = True

flag_experiment     = True

flag_model_load     = False
flag_model_save     = True

flag_EarlyStopping  = True
flag_es_monitor     = "val_loss"
#flag_es_monitor     = "val_accuracy"
### ---------- ---------- ---------- ---------- ----------
### Pre-processing
# Ratio of training dataset to be split
ratio_train = 0.85

# Randam seed for reproducibility
seed = 7

# scaling
flag_scaling        = "Std" # for Gaussian
#flag_scaling        = "Norm" # (0 - 1)

# Sliding window
window_size = 15
window_step = 8

flag_sw_label       = "last"
#flag_sw_label       = "mode"

# Data balancing
flag_balance_under1 = False
flag_balance_under2 = False
flag_balance_under3 = False

flag_balance_over1  = False
flag_balance_over2  = False
flag_balance_over3  = False
### ---------- ---------- ---------- ---------- ----------
### Evaluation
flag_data_dist      = False
flag_interpolate    = True
flag_plot_model     = True
flag_savefig        = True

flag_summary        = True
flag_cm_norm        = True

flag_TensorBoard    = False
### ---------- ---------- ---------- ---------- ----------
### Directories
dir_log = 'log'
dir_model = 'model'
### ---------- ---------- ---------- ---------- ----------
### Names
# models
modelname_cnn_1d         = 'CNN_1D'
modelname_lstm_Mto1      = 'LSTM_Mto1'
modelname_cnn1d_lstm     = 'CNN1D_LSTM'
modelname_convlstm       = 'ConvLSTM'
modelname_ensemble       = 'Ensemble'
modelname_lstm_Mto1_null = 'LSTM_Mto1_null'

# Label list
labels_Loco = ['(Null)',
               'Stand',
               'Walk',
               'Sit',
               'Lie']

labels_ML = ['(Null)',
             'Open Door 1',     'Open Door 2',
             'Close Door 1',    'Close Door 2',
             'Open Fridge',     'Close Fridge',
             'Open Dishwasher', 'Close Dishwasher',
             'Open Drawer 1',   'Close Drawer 1',
             'Open Drawer 2',   'Close Drawer 2',
             'Open Drawer 3',   'Close Drawer 3',
             'Clean Table',     'Drink from Cup', 'Toggle Switch']
### ---------- ---------- ---------- ---------- ----------

# 2.Setup

## Import libraries

In [ ]:
# Pre-process
import os
import glob
import numpy as np
import random as rn
import pandas as pd
from numpy.lib.stride_tricks import as_strided
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.utils import to_categorical
import collections

# Evaluation
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import time
from datetime import datetime
from numpy import mean, std
from matplotlib import pyplot
import seaborn as sns
sns.set()

# NNs
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# CNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Input, BatchNormalization
from tensorflow.keras.optimizers import Adam

# LSTM
from tensorflow.keras.layers import LSTM, TimeDistributed
from tensorflow.keras import regularizers

# ConvLSTM
from tensorflow.keras.layers import ConvLSTM2D

# Ensemble
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate

In [ ]:
# Set random seed (for reproducibility)
# Hash seed
os.environ["PYTHONHASHSEED"] = str(seed)

# Built-in random
rn.seed(seed)

# Numpy.random
np.random.seed(seed)

# Tensorflow
tf.random.set_seed(seed)

## [F] Pre-processing

In [ ]:
def read_files(files):
    for i, file in enumerate(files):
        print(f'[{i+1}] Reading file: {file}')
        d = pd.read_csv(file, header=None, sep=' ')

        # Truncate last residual records for sliding window
        mod = (len(d) - window_size) % window_step
        d = d[:len(d)-mod]

        # Count records with NaN
        d_nan = d.isnull().sum()

        # Convert NaN
        # Linear interpolation
        if flag_interpolate:
            d.interpolate(inplace=True)

        # Convert remaining NaNs into 0
        if flag_interpolate:
            d.replace(np.nan, 0, inplace=True)

        if i == 0:
            dataset = d
            dataset_nan = d_nan
        else:
            dataset = pd.concat([dataset, d])
            dataset_nan = dataset_nan + d_nan
            
    return dataset, dataset_nan

In [ ]:
# Adjust label values (0 to num_classes)
def adjust_idx_labels(data_y):
    if flag_label == 'Locomotion':
        data_y[data_y == 4] = 3
        data_y[data_y == 5] = 4
    elif flag_label == 'ML_Both_Arms':
        data_y[data_y == 406516] = 1
        data_y[data_y == 406517] = 2
        data_y[data_y == 404516] = 3
        data_y[data_y == 404517] = 4
        data_y[data_y == 406520] = 5
        data_y[data_y == 404520] = 6
        data_y[data_y == 406505] = 7
        data_y[data_y == 404505] = 8
        data_y[data_y == 406519] = 9
        data_y[data_y == 404519] = 10
        data_y[data_y == 406511] = 11
        data_y[data_y == 404511] = 12
        data_y[data_y == 406508] = 13
        data_y[data_y == 404508] = 14
        data_y[data_y == 408512] = 15
        data_y[data_y == 407521] = 16
        data_y[data_y == 405506] = 17

    return data_y

In [ ]:
def sliding_window(data, w_size, w_step):
    shape = np.array(data.shape)

    # Compute new shape & strides based on window size & step
    newshape = ((shape - w_size) // w_step) + 1
    newshape = np.append(newshape, [w_size[0], data.shape[1]])

    # Original strides * window step
    newstrides = np.array(data.strides) * w_step

    # For window size & features, set original strides
    newstrides = np.append(newstrides, data.strides)

    # Create a view for new shape & stride
    data_strided = as_strided(data, shape=newshape, strides=newstrides)

    # Flatten strided shape
    newshape_flatten = [i for i in newshape if i != 1]

    return data_strided.reshape(newshape_flatten)

In [ ]:
def opp_sliding_window(X, Y):
    X = sliding_window(X, (window_size, X.shape[1]), (window_step, 1))
    Y = sliding_window(Y, (window_size, Y.shape[1]), (window_step, 1))

    return X, Y

## [F] ML models

In [ ]:
# Train
epochs = 100
batch_size = 100
repeats = 10

# EarlyStopping
es_patience = 5

### CNN

In [ ]:
# Layer
cnn_padding ='same'
cnn_activation = 'relu'
cnn_units = 128
cnn_dropout = 0.5
cnn_pool_size = 2

## 1D Conv
cnn_1d_filters = 64
cnn_1d_kernel_size = 5

In [ ]:
def build_model_cnn_1d():
    model = Sequential(name=modelname_cnn_1d)

    # Conv layer 1
    model.add(Conv1D(
        input_shape = cnn_1d_input_shape,
        filters     = cnn_1d_filters,
        kernel_size = cnn_1d_kernel_size,
        padding     = cnn_padding,
        activation  = cnn_activation))

    # Conv layer 2
    model.add(Conv1D(
        filters     = cnn_1d_filters,
        kernel_size = cnn_1d_kernel_size,
        padding     = cnn_padding,
        activation  = cnn_activation))

    # Conv layer 3
    model.add(Conv1D(
        filters     = cnn_1d_filters,
        kernel_size = cnn_1d_kernel_size,
        padding     = cnn_padding,
        activation  = cnn_activation))

    # Conv layer 4
    model.add(Conv1D(
        filters     = cnn_1d_filters,
        kernel_size = cnn_1d_kernel_size,
        padding     = cnn_padding,
        activation  = cnn_activation))

    # Maxpool layer
    # model.add(MaxPool1D(
    #     pool_size = cnn_pool_size))

    model.add(Flatten())

    # Dense layer 1
    model.add(Dense(
        units      = cnn_units,
        activation = 'relu'))

    # Dropout
    model.add(Dropout(cnn_dropout))

    # Dense layer 2
    model.add(Dense(
        units      = cnn_units,
        activation = 'relu'))

    # Dropout
    model.add(Dropout(cnn_dropout))

    # Output layer
    model.add(Dense(
        units      = num_classes,
        activation = 'softmax'))

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

### LSTM

In [ ]:
# Layer
lstm_units = 128
lstm_dropout = 0.5
lstm_weight_decay = 1e-4

In [ ]:
# LSTM (Many-to-One, stateless)
def build_model_lstm_Mto1():
    model = Sequential(name=modelname_lstm_Mto1)

    # LSTM layer
    model.add(LSTM(
        input_shape        = lstm_input_shape,
        units              = lstm_units,
        # kernel_regularizer = regularizers.l2(lstm_weight_decay),
        return_sequences   = False)) # final layer of LSTM (only final output)

    # Dropout
    model.add(Dropout(lstm_dropout))

    # Dense layer
    model.add(Dense(
        units      = lstm_units,
        activation = 'relu'))

    # Dropout
    model.add(Dropout(lstm_dropout))

    # Output layer
    model.add(Dense(
        units      = num_classes,
        activation = 'softmax'))

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

### CNN-LSTM

In [ ]:
# Data
cnn_lstm_steps = 3
cnn_lstm_length = int(window_size / cnn_lstm_steps)

# Layer
cnn_lstm_padding = 'same'
cnn_lstm_activation = 'relu'
cnn_lstm_dropout = 0.5
cnn_lstm_pool_size = 2

## CNN
cnn_lstm_filters = 64
cnn1d_lstm_kernel_size = 3

# LSTM
cnn_lstm_units = 128

In [ ]:
def build_model_cnn1d_lstm():
    model = Sequential(name=modelname_cnn1d_lstm)

    ## CNN (with TimeDistributed)
    # Conv layer 1
    model.add(TimeDistributed(Conv1D(
        filters     = cnn_lstm_filters,
        kernel_size = cnn1d_lstm_kernel_size,
        padding     = cnn_lstm_padding,
        activation  = cnn_lstm_activation),
        input_shape = cnn1d_lstm_input_shape))

    # Conv layer 2
    model.add(TimeDistributed(Conv1D(
        filters     = cnn_lstm_filters,
        kernel_size = cnn1d_lstm_kernel_size,
        padding     = cnn_lstm_padding,
        activation  = cnn_lstm_activation)))

    # Conv layer 3
    model.add(TimeDistributed(Conv1D(
        filters     = cnn_lstm_filters,
        kernel_size = cnn1d_lstm_kernel_size,
        padding     = cnn_lstm_padding,
        activation  = cnn_lstm_activation)))

    # Conv layer 4
    model.add(TimeDistributed(Conv1D(
        filters     = cnn_lstm_filters,
        kernel_size = cnn1d_lstm_kernel_size,
        padding     = cnn_lstm_padding,
        activation  = cnn_lstm_activation)))

    # Dropout
    model.add(TimeDistributed(Dropout(cnn_lstm_dropout)))

    # Maxpool layer
    model.add(TimeDistributed(MaxPool1D(
        pool_size = cnn_lstm_pool_size)))

    model.add(TimeDistributed(Flatten()))

    ## LSTM
    # LSTM layer 1
    model.add(LSTM(
        units            = cnn_lstm_units,
        return_sequences = True))

    # Dropout
    model.add(Dropout(cnn_lstm_dropout))

    # LSTM layer 2
    model.add(LSTM(
        units            = cnn_lstm_units,
        return_sequences = False))

    # Dropout
    model.add(Dropout(cnn_lstm_dropout))

    # Output layer
    model.add(Dense(
        units      = num_classes,
        activation = 'softmax'))

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

### ConvLSTM

In [ ]:
# Data
convlstm_steps = 3
convlstm_length = int(window_size / convlstm_steps)

# Layer
convlstm_padding = 'same'
convlstm_activation = 'relu'
convlstm_dropout = 0.5
convlstm_pool_size = 2

## CNN
convlstm_filters = 64
convlstm_kernel_size = (1, 3)

convlstm_units = 128

In [ ]:
def build_model_convlstm():
    model = Sequential(name=modelname_convlstm)

    # Conv LSTM layer 1
    model.add(ConvLSTM2D(
        filters          = convlstm_filters,
        kernel_size      = convlstm_kernel_size,
        padding          = convlstm_padding,
        activation       = convlstm_activation,
        input_shape      = convlstm_input_shape,
        return_sequences = True))
        # return_sequences = False)) # final layer of LSTM (only final output)

    # Conv LSTM layer 2
    model.add(ConvLSTM2D(
        filters          = convlstm_filters,
        kernel_size      = convlstm_kernel_size,
        padding          = convlstm_padding,
        activation       = convlstm_activation,
        return_sequences = True))

    # Conv LSTM layer 3
    model.add(ConvLSTM2D(
        filters          = convlstm_filters,
        kernel_size      = convlstm_kernel_size,
        padding          = convlstm_padding,
        activation       = convlstm_activation,
        return_sequences = True))

    # Conv LSTM layer 4
    model.add(ConvLSTM2D(
        filters          = convlstm_filters,
        kernel_size      = convlstm_kernel_size,
        padding          = convlstm_padding,
        activation       = convlstm_activation,
        return_sequences = False))

    # Dropout
    model.add(Dropout(convlstm_dropout))

    model.add(Flatten())

    # Dense layer
    model.add(Dense(
        units      = convlstm_units,
        activation = convlstm_activation))

    # Output layer
    model.add(Dense(
        units      = num_classes,
        activation = 'softmax'))

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

### Ensemble

In [ ]:
# Layer
ensemble_units = 10
ensemble_activation = 'relu'

In [ ]:
def build_model_ensemble(inputs, outputs):
    ensemble_merge = Concatenate(axis=1)(outputs)

    # Dense layer
    ensemble_hidden = Dense(
        units      = ensemble_units,
        activation = ensemble_activation)(ensemble_merge)

    # Output layer
    ensemble_output = Dense(
        units      = num_classes,
        activation = 'softmax')(ensemble_hidden)

    model = Model(
        inputs     = inputs,
        outputs    = ensemble_output,
        name       = modelname_ensemble)

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

## [F] Evaluation

In [ ]:
# Train and evaluate a model (once)
def evaluate_model(model_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Build model
    if model_name == modelname_cnn_1d:
        model = build_model_cnn_1d()
    elif model_name == modelname_lstm_Mto1:
        model = build_model_lstm_Mto1()
    elif model_name == modelname_cnn1d_lstm:
        model = build_model_cnn1d_lstm()
    elif model_name == modelname_convlstm:
        model = build_model_convlstm()
    else:
        print("Error: specify correct model name")
        return -1
    
    # Train
    history = model.fit(
        x               = X_train,
        y               = y_train,
        batch_size      = batch_size,
        epochs          = epochs,
        verbose         = 0,
        callbacks       = [cb],
        validation_data = (X_val, y_val)
    )

    num_epochs = len(history.history['loss'])

    ## Evaluate
    # Accuracy
    _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

    # F1
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test.argmax(axis=-1), y_pred.argmax(axis=-1), average='weighted')

    return accuracy, f1, num_epochs

In [ ]:
# Repeat experiment
def run_experiment(model_name, X_train, X_val, X_test, y_train, y_val, y_test, repeats=10):
    print(f'Model: {model_name}')

    scores_acc = []
    scores_f1 = []
    scores_epoch = []
    for r in range(repeats):
        acc, f1, epoch = evaluate_model(model_name, X_train, y_train, X_val, y_val, X_test, y_test)
        print(f'[#{r+1:>2d}] Accuracy: {acc:.3f}, F1 score(weighted): {f1:.3f}, epoch: {epoch}')
        scores_acc.append(acc)
        scores_f1.append(f1)
        scores_epoch.append(epoch)
    
    # Summarise mean and standard deviation
    print(f'Accuracy: {mean(scores_acc):.3f} (+/- {std(scores_acc):.3f})')
    print(f'F1 score(weighted): {mean(scores_f1):.3f} (+/- {std(scores_f1):.3f})')
    print(f'epoch: {mean(scores_epoch):.1f} (+/- {std(scores_epoch):.3f})')

	# Boxplot of scores
    metrics_list = ['Accuracy', 'F1 score']
    all_scores = []
    all_scores.append(scores_acc)
    all_scores.append(scores_f1)
    plt.boxplot(all_scores, labels=metrics_list)

    if flag_savefig:
        plt.savefig("boxplot_" + model_name + ".png")
        
    plt.show()

In [ ]:
# Plot a histogram of each variable in the dataset
def plot_variable_distributions(X, start=0, end=None, xlim=None):
    if end is None:
        end = X.shape[1]-1

    print(X.shape)
    num_features = end - start +1
    print(f'# of plots: {num_features} ({start} - {end})')

    plt.figure(figsize=(10, 2*num_features), tight_layout=True)
    xaxis = None
    for i, f in enumerate(range(start, end+1)):
        print(i)
        if xlim is None:
            ax = plt.subplot(num_features, 1, i+1, title='Feature: ' + str(f))
        else:
            ax = plt.subplot(num_features, 1, i+1, sharex=xaxis, title='Feature: ' + str(f))
            ax.set_xlim(xlim)
        if i == 0:
            xaxis = ax
        plt.hist(X[:, f], bins=100)
    plt.show()

In [ ]:
# Plot graphs for loss and accuracy
def plot_acc_graph(history):

    # Set figure size
    fig = plt.figure(figsize=(15, 6))
    plt.subplots_adjust(wspace=0.2)

    # Loss
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'],
            label='Train',
            color='black')

    plt.plot(history.history['val_loss'],
            label='Val',
            color='red')

    #plt.ylim(0, 1)
    plt.legend()
    plt.grid(True)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    # Accuracy
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'],
            label='Train',
            color='black')

    plt.plot(history.history['val_accuracy'],
            label='Val',
            color='red')

    plt.ylim(0, 1)
    plt.legend()
    plt.grid(True)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')

    if flag_savefig:
    	plt.savefig("acc_graph_" + history.model.name + ".png")

    plt.show()

In [ ]:
# Print execution time
def print_execution_time(time_start):
    time_elapsed = time.perf_counter() - time_start
    min, sec = divmod(time_elapsed, 60)
    hour, min = divmod(min, 60)

    print(f"Execution time: {hour:.0f} hour {min:.0f} min {sec:.0f} sec")

# 3.Pre-processing

In [ ]:
# For CSF3 (UoM) setting
import platform
system_name = platform.system()
print('system_name: ' + system_name)
if system_name == "Linux":
    flag_summary    = False
    flag_cm_norm    = False
    flag_plot_model = False

## Read files

In [ ]:
# dataset files (train & test)
files_train = glob.glob('../data/train/*.dat')
files_test = glob.glob('../data/test/*.dat')

# Read datafiles (if not yet)
if not 'R_dataset_train' in locals():
    R_dataset_train, nan_train = read_files(files_train)
    R_dataset_test, nan_test = read_files(files_test)

In [ ]:
# Discard null action records
if flag_delete_null:
    if flag_label == 'Locomotion':
        dataset_train = R_dataset_train[R_dataset_train.iloc[:, 243] != 0]
        dataset_test = R_dataset_test[R_dataset_test.iloc[:, 243] != 0]
    elif flag_label == 'ML_Both_Arms':
        dataset_train = R_dataset_train[R_dataset_train.iloc[:, 249] != 0]
        dataset_test = R_dataset_test[R_dataset_test.iloc[:, 249] != 0]
else:
    dataset_train = R_dataset_train
    dataset_test = R_dataset_test

In [ ]:
# Balancing data 1 (After reading files)
if flag_balance_under1:
    if flag_label == 'Locomotion':
        idx_label = 243
    elif flag_label == 'ML_Both_Arms':
        idx_label = 249

    min_train = dataset_train.iloc[:, idx_label].value_counts().min()
    dataset_train_np = dataset_train.to_numpy()

    for i in dataset_train.iloc[:, idx_label].unique():
        dataset_train_np = np.delete(dataset_train_np, np.where(dataset_train_np[:, idx_label] == i)[0][min_train:], axis=0)

    dataset_train = pd.DataFrame(dataset_train_np)

## Divide X / Y
(features and labels)


In [ ]:
## Features (X)
# Strip unnecessay columns
# (following opportunity challenge specification)
X_train = pd.concat([
    dataset_train.iloc[:, 1:46], # (included:excluded)
    dataset_train.iloc[:, 50:59],
    dataset_train.iloc[:, 63:72],
    dataset_train.iloc[:, 76:85],
    dataset_train.iloc[:, 89:98],
    dataset_train.iloc[:, 102:134]],
    axis=1)

X_test = pd.concat([
    dataset_test.iloc[:, 1:46],
    dataset_test.iloc[:, 50:59],
    dataset_test.iloc[:, 63:72],
    dataset_test.iloc[:, 76:85],
    dataset_test.iloc[:, 89:98],
    dataset_test.iloc[:, 102:134]],
    axis=1)

## Labels (Y)
# from last 7 columns
if flag_label == 'Locomotion':
    y_train = dataset_train.iloc[:,243]
    y_test = dataset_test.iloc[:,243]
elif flag_label == 'ML_Both_Arms':
    y_train = dataset_train.iloc[:,249]
    y_test = dataset_test.iloc[:,249]

y_train = y_train.rename('Label')
y_test = y_test.rename('Label')

In [ ]:
num_features = len(X_train.columns)

# Input shape of NNs
cnn_1d_input_shape = (window_size, num_features)
lstm_input_shape = (window_size, num_features)
cnn1d_lstm_input_shape = (None, cnn_lstm_length, num_features)
convlstm_input_shape = (convlstm_steps, 1, convlstm_length, num_features)

## (Distributions)

In [ ]:
if flag_data_dist:
    plot_variable_distributions(X_train.to_numpy(), start=0, end=29)
    plot_variable_distributions(X_train.to_numpy(), start=30, end=59)
    plot_variable_distributions(X_train.to_numpy(), start=60, end=89)
    plot_variable_distributions(X_train.to_numpy(), start=90, end=112)

## Encode labels

In [ ]:
## Encode label (one-hot)
# Adjust label values for to_categorical()
y_train = adjust_idx_labels(y_train)
y_test = adjust_idx_labels(y_test)

# Convert class vector (int) to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

## Split train / val

In [ ]:
# Split into train and val (No shuffle)
X_train, X_val, y_train, y_val = \
    train_test_split(X_train, y_train,
                     train_size=ratio_train,
                     random_state=seed,
                     shuffle=False)

In [ ]:
# Balancing data 2 (After splitting train, val, and test)
if flag_balance_under2:
    min_train = y_train.value_counts().min()

    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()

    y_train_n = y_train.argmax(axis=-1)

    for i in range(len(np.unique(y_train_n))):
        X_train = np.delete(X_train, np.where(y_train_n == i)[0][min_train:], axis=0)
        y_train_n = np.delete(y_train_n, np.where(y_train_n == i)[0][min_train:], axis=0)

    y_train = to_categorical(y_train_n)

    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)

In [ ]:
# The number of classes
num_classes = len(y_train.columns)

# label list (for classification_report, confusion_matrix)
if flag_label == 'Locomotion':
    labels_cr = labels_Loco
    labels_cm = labels_Loco
elif flag_label == 'ML_Both_Arms':
    labels_cr = labels_ML
    labels_cm = labels_ML

if flag_delete_null:
     labels_cr = np.delete(labels_cr, 0)
#     labels_cm = np.delete(labels_cm, 0)

# confusion_matrix
labels = np.arange(0, num_classes)

## Scaling


In [ ]:
if flag_scaling == "Norm":
    scaler = MinMaxScaler()
elif flag_scaling == "Std":
    scaler = StandardScaler()

# Fit the scaler on the training data (to avoid data leakage)
scaler.fit(X_train)

# Scale (to numpy)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Convert to numpy
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_test = y_test.to_numpy()

## (Distributions)

In [ ]:
if flag_data_dist:
    plot_variable_distributions(X_train, start=0, end=29)
    plot_variable_distributions(X_train, start=30, end=59)
    plot_variable_distributions(X_train, start=60, end=89)
    plot_variable_distributions(X_train, start=90, end=112)

## Sliding window

In [ ]:
X_train_sw, y_train_sw = opp_sliding_window(X_train, y_train)
X_val_sw, y_val_sw  = opp_sliding_window(X_val, y_val)
X_test_sw, y_test_sw  = opp_sliding_window(X_test, y_test)

if flag_sw_label == "last":
    # last class of each sliding window
    y_train_sw_label = np.asarray([[i[-1]] for i in y_train_sw]).reshape(-1, y_train_sw.shape[-1])
    y_val_sw_label = np.asarray([[i[-1]] for i in y_val_sw]).reshape(-1, y_val_sw.shape[-1])
    y_test_sw_label = np.asarray([[i[-1]] for i in y_test_sw]).reshape(-1, y_test_sw.shape[-1])

elif flag_sw_label == "mode":
    # mode in each sliding window
    y_train_sw_mode = np.asarray([collections.Counter(i.argmax(axis=-1)).most_common()[0][0] for i in y_train_sw])
    y_train_sw_label = to_categorical(y_train_sw_mode)
    y_val_sw_mode = np.asarray([collections.Counter(i.argmax(axis=-1)).most_common()[0][0] for i in y_val_sw])
    y_val_sw_label = to_categorical(y_val_sw_mode)
    y_test_sw_mode = np.asarray([collections.Counter(i.argmax(axis=-1)).most_common()[0][0] for i in y_test_sw])
    y_test_sw_label = to_categorical(y_test_sw_mode)

# For evaluation
y_test_classes_sw_label = y_test_sw_label.argmax(axis=-1)

In [ ]:
# Blancing data 3 (After sliding window)
if flag_balance_under3:
    y_train_sw_n = y_train_sw_label.argmax(axis=-1)
    min_train = pd.DataFrame(y_train_sw_label).value_counts().min()

    for i in range(num_classes):
        X_train_sw = np.delete(X_train_sw, np.where(y_train_sw_n == i)[0][min_train:], axis=0)
        y_train_sw_n = np.delete(y_train_sw_n, np.where(y_train_sw_n == i)[0][min_train:], axis=0)

    y_train_sw_label = to_categorical(y_train_sw_n)

elif flag_balance_over3:
    y_train_sw_n = y_train_sw_label.argmax(axis=-1)
    max_train = pd.DataFrame(y_train_sw_n)[0].value_counts().max()
    num_labels = np.unique(y_train_sw_n).size

    X_train_sw_balanced = np.empty((num_labels * max_train, X_train_sw.shape[1], X_train_sw.shape[2]))
    y_train_sw_balanced = np.empty((num_labels * max_train, y_train_sw.shape[1], y_train_sw.shape[2]))
    y_train_sw_label_balanced = np.empty((num_labels * max_train, y_train_sw_label.shape[1]))

    X_train_sw_balanced[:X_train_sw.shape[0]] = X_train_sw
    y_train_sw_balanced[:y_train_sw.shape[0]] = y_train_sw
    y_train_sw_label_balanced[:y_train_sw_label.shape[0]] = y_train_sw_label

    l = X_train_sw.shape[0]
    for c in np.unique(y_train_sw_n):
        num = np.count_nonzero(y_train_sw_n == c)
        if max_train > num:
            num_diff = max_train - num
            idx_c = np.where(y_train_sw_n == c)[0]
            idx_add = np.random.choice(idx_c, num_diff, replace=True)        
            for i in idx_add:
                X_train_sw_balanced[l] = X_train_sw[i]
                y_train_sw_balanced[l] = y_train_sw[i]
                y_train_sw_label_balanced[l] = y_train_sw_label[i]
                l += 1

    X_train_sw = X_train_sw_balanced
    y_train_sw = y_train_sw_balanced
    y_train_sw_label = y_train_sw_label_balanced

## [Summary]

In [ ]:
if flag_summary:
    # The number of samples (train, val, test)
    num_samples = len(X_train) + len(X_val) + len(X_test)
    num_train = X_train.shape[0]
    num_val = X_val.shape[0]
    num_test = X_test.shape[0]

    num_classes_train = y_train.shape[-1]
    num_classes_val = y_val.shape[-1]
    num_classes_test = y_test.shape[-1]

    y_counts = pd.concat([np.flip(pd.DataFrame(y_train).value_counts(sort=False)),
                        np.flip(pd.DataFrame(y_val).value_counts(sort=False)),
                        np.flip(pd.DataFrame(y_test).value_counts(sort=False))],
                        axis=1)

    y_counts = y_counts.style.hide_index().highlight_null('red').set_precision(0)

    y_counts_sw = pd.concat([np.flip(pd.DataFrame(y_train_sw_label).value_counts(sort=False)),
                        np.flip(pd.DataFrame(y_val_sw_label).value_counts(sort=False)),
                        np.flip(pd.DataFrame(y_test_sw_label).value_counts(sort=False))],
                    axis=1)

    y_counts_sw = y_counts_sw.style.hide_index().highlight_null('red').set_precision(0)

    print('[# of samples]')
    print(f'Total: {num_samples:>7,}')
    print(f'Train: {num_train:>7,}')
    print(f'Val:   {num_val:>7,}')
    print(f'Test:  {num_test:>7,}')
    print()

    print('[After sliding window]')
    print(f'Train: {X_train_sw.shape[0]:>7,}')
    print(f'Val:   {X_val_sw.shape[0]:>7,}')
    print(f'Test:  {X_test_sw.shape[0]:>7,}')
    print()

    print('[# of features]')
    print(f'{num_features}')
    print()

    print('[# of classes]')
    print(f'Total: {num_classes}')
    print(f'Train: {num_classes_train}')
    print(f'Val:   {num_classes_val}')
    print(f'Test:  {num_classes_test}')
    print()

    print('[Original data]')
    print('Train   Val   Test')
    display(y_counts)
    print()

    print('[After sliding window]')
    print('Train   Val   Test')
    display(y_counts_sw)

# 4.Train & Test


In [ ]:
## callbacks
cb = []

# EarlyStopping
if flag_EarlyStopping:
    es = EarlyStopping(monitor=flag_es_monitor, patience=es_patience)
    cb.append(es)

# TensorBoard
if flag_TensorBoard:
    log_dir = dir_log + '/tb/' + datetime.utcnow().strftime('%Y%m%d-%H%M%S')
    tb = TensorBoard(log_dir=log_dir, histogram_freq=1)
    cb.append(tb)

In [ ]:
# Train, val, test sets
X_train = X_train_sw
X_val = X_val_sw
X_test = X_test_sw

y_train = y_train_sw_label
y_val = y_val_sw_label
y_test = y_test_sw_label

y_test_classes = y_test_classes_sw_label

## CNN


In [ ]:
if flag_CNN_1d:
    if flag_model_load:
        model_cnn_1d = load_model(dir_model + "/" + modelname_cnn_1d + ".h5")
    else:
        model_cnn_1d = build_model_cnn_1d()
    model_cnn_1d.summary()

    if flag_plot_model:
        plot_model(model_cnn_1d, show_shapes=True, to_file='model_' + model_cnn_1d.name + '.png')

In [ ]:
if flag_CNN_1d:
    if not flag_model_load:
        time_start = time.perf_counter()

        history_cnn_1d = model_cnn_1d.fit(
            x               = X_train,
            y               = y_train,
            batch_size      = batch_size,
            epochs          = epochs,
            verbose         = 1,
            callbacks       = [cb],
            validation_data = (X_val, y_val)
        )

        print_execution_time(time_start)

In [ ]:
if flag_CNN_1d:
    if not flag_model_load:
        plot_acc_graph(history_cnn_1d)

In [ ]:
if flag_CNN_1d:
    if flag_model_save:
        time_current = datetime.utcnow().strftime('%Y%m%d-%H%M%S')
        model_cnn_1d.save(dir_model + '/' + model_cnn_1d.name + '_' + time_current + ".h5", overwrite=False)

In [ ]:
if flag_CNN_1d:
    y_pred_classes = model_cnn_1d.predict(X_test).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.3f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_cr))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_cm, columns=labels_cm)
    plt.figure(figsize=(10,10))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    if flag_savefig:
    	plt.savefig("confusion_matrix_" + model_cnn_1d.name + ".png")

    plt.plot()

## LSTM

In [ ]:
if flag_LSTM_Mto1:
    if flag_model_load:
        model_lstm_Mto1 = load_model(dir_model + "/" + modelname_lstm_Mto1 + ".h5")
    else:
        model_lstm_Mto1 = build_model_lstm_Mto1()
    model_lstm_Mto1.summary()

    if flag_plot_model:
        plot_model(model_lstm_Mto1, show_shapes=True, to_file='model_' + model_lstm_Mto1.name + '.png')

In [ ]:
if flag_LSTM_Mto1:
    if not flag_model_load:
        time_start = time.perf_counter()

        history_lstm_Mto1 = model_lstm_Mto1.fit(
            X_train,
            y_train,
            batch_size      = batch_size,
            epochs          = epochs,
            verbose         = 1,
            callbacks       = [cb],
            validation_data = (X_val, y_val)
        )

        print_execution_time(time_start)

In [ ]:
if flag_LSTM_Mto1:
    if not flag_model_load:
        plot_acc_graph(history_lstm_Mto1)

In [ ]:
if flag_LSTM_Mto1:
    if flag_model_save:
        time_current = datetime.utcnow().strftime('%Y%m%d-%H%M%S')
        model_lstm_Mto1.save(dir_model + '/' + model_lstm_Mto1.name + '_' + time_current + ".h5", overwrite=False)

In [ ]:
if flag_LSTM_Mto1:
    y_pred_classes = model_lstm_Mto1.predict(X_test).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.3f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_cr))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_cm, columns=labels_cm)
    plt.figure(figsize=(10,10))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    if flag_savefig:
    	plt.savefig("confusion_matrix_" + model_lstm_Mto1.name + ".png")

    plt.plot()

## CNN-LSTM

In [ ]:
# Reshape data into 4D for CNN1D-LSTM
X_train_cnn1d_lstm = X_train.reshape((X_train.shape[0], cnn_lstm_steps, cnn_lstm_length, X_train.shape[-1]))
X_val_cnn1d_lstm = X_val.reshape((X_val.shape[0], cnn_lstm_steps, cnn_lstm_length, X_val.shape[-1]))
X_test_cnn1d_lstm = X_test.reshape((X_test.shape[0], cnn_lstm_steps, cnn_lstm_length, X_test.shape[-1]))

In [ ]:
if flag_CNN1D_LSTM:
    if flag_model_load:
        model_cnn1d_lstm = load_model(dir_model + "/" + modelname_cnn1d_lstm + ".h5")
    else:
        model_cnn1d_lstm = build_model_cnn1d_lstm()
    model_cnn1d_lstm.summary()

    if flag_plot_model:
        plot_model(model_cnn1d_lstm, show_shapes=True, to_file='model_' + model_cnn1d_lstm.name + '.png')

In [ ]:
if flag_CNN1D_LSTM:
    if not flag_model_load:
        time_start = time.perf_counter()

        history_cnn1d_lstm = model_cnn1d_lstm.fit(
            X_train_cnn1d_lstm,
            y_train,
            batch_size      = batch_size,
            epochs          = epochs,
            verbose         = 1,
            callbacks       = [cb],
            validation_data = (X_val_cnn1d_lstm, y_val)
        )

        print_execution_time(time_start)

In [ ]:
if flag_CNN1D_LSTM:
    if not flag_model_load:
        plot_acc_graph(history_cnn1d_lstm)

In [ ]:
if flag_CNN1D_LSTM:
    if flag_model_save:
        time_current = datetime.utcnow().strftime('%Y%m%d-%H%M%S')
        model_cnn1d_lstm.save(dir_model + '/' + model_cnn1d_lstm.name + '_' + time_current + ".h5", overwrite=False)

In [ ]:
if flag_CNN1D_LSTM:
    y_pred_classes = model_cnn1d_lstm.predict(X_test_cnn1d_lstm).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.3f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_cr))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_cm, columns=labels_cm)
    plt.figure(figsize=(10,10))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    if flag_savefig:
    	plt.savefig("confusion_matrix_" + model_cnn1d_lstm.name + ".png")

    plt.plot()

## ConvLSTM

In [ ]:
# Reshape data into 5D for ConvLSTM
X_train_convlstm = X_train.reshape((X_train.shape[0], convlstm_steps, 1, convlstm_length, X_train.shape[-1]))
X_val_convlstm = X_val.reshape((X_val.shape[0], convlstm_steps, 1, convlstm_length, X_val.shape[-1]))
X_test_convlstm = X_test.reshape((X_test.shape[0], convlstm_steps, 1, convlstm_length, X_test.shape[-1]))

In [ ]:
if flag_ConvLSTM:
    if flag_model_load:
        model_convlstm = load_model(dir_model + "/" + modelname_convlstm + ".h5")
    else:
        model_convlstm = build_model_convlstm()
    model_convlstm.summary()

    if flag_plot_model:
        plot_model(model_convlstm, show_shapes=True, to_file='model_' + model_convlstm.name + '.png')

In [ ]:
if flag_ConvLSTM:
    if not flag_model_load:
        time_start = time.perf_counter()

        history_convlstm = model_convlstm.fit(
            X_train_convlstm,
            y_train,
            batch_size      = batch_size,
            epochs          = epochs,
            verbose         = 1,
            callbacks       = [cb],
            validation_data = (X_val_convlstm, y_val)
        )

        print_execution_time(time_start)

In [ ]:
if flag_ConvLSTM:
    if not flag_model_load:
        plot_acc_graph(history_convlstm)

In [ ]:
if flag_ConvLSTM:
    if flag_model_save:
        time_current = datetime.utcnow().strftime('%Y%m%d-%H%M%S')
        model_convlstm.save(dir_model + '/' + model_convlstm.name + '_' + time_current + ".h5", overwrite=False)

In [ ]:
if flag_ConvLSTM:
    y_pred_classes = model_convlstm.predict(X_test_convlstm).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.3f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_cr))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_cm, columns=labels_cm)
    plt.figure(figsize=(10,10))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    if flag_savefig:
    	plt.savefig("confusion_matrix_" + model_convlstm.name + ".png")

    plt.plot()

## Ensemble

In [ ]:
if flag_Ensemble:
    # build sub-models
    sub_models = list()
    X_train_ensemble = list()
    X_val_ensemble = list()
    X_test_ensemble = list()

    if flag_CNN_1d:
        sub_models.append(model_cnn_1d)
        X_train_ensemble.append(X_train)
        X_val_ensemble.append(X_val)
        X_test_ensemble.append(X_test)
    if flag_LSTM_Mto1:
        sub_models.append(model_lstm_Mto1)
        X_train_ensemble.append(X_train)
        X_val_ensemble.append(X_val)
        X_test_ensemble.append(X_test)
    if flag_CNN1D_LSTM:
        sub_models.append(model_cnn1d_lstm)
        X_train_ensemble.append(X_train_cnn1d_lstm)
        X_val_ensemble.append(X_val_cnn1d_lstm)
        X_test_ensemble.append(X_test_cnn1d_lstm)
    if flag_ConvLSTM:
        sub_models.append(model_convlstm)
        X_train_ensemble.append(X_train_convlstm)
        X_val_ensemble.append(X_val_convlstm)
        X_test_ensemble.append(X_test_convlstm)

    # freeze parameters for sub-models
    for i in range(len(sub_models)):
        for layer in sub_models[i].layers:
            layer.trainable = False
            layer._name = 'ensemble_' + sub_models[i].name + '_' + layer.name

    inputs = [model.input for model in sub_models]
    outputs = [model.output for model in sub_models]

    model_ensemble = build_model_ensemble(inputs, outputs)
    model_ensemble.summary()

    if flag_plot_model:
        plot_model(model_ensemble, show_shapes=True, to_file='model_' + model_ensemble.name + '.png')

In [ ]:
if flag_Ensemble:
    history_ensemble    = model_ensemble.fit(
        x               = X_train_ensemble,
        y               = y_train,
        batch_size      = batch_size,
        epochs          = epochs,
        verbose         = 1,
        callbacks       = [cb],
        validation_data = (X_val_ensemble, y_val)
    )

In [ ]:
if flag_Ensemble:
    plot_acc_graph(history_ensemble)

In [ ]:
if flag_Ensemble:
    y_pred_classes = model_ensemble.predict(X_test_ensemble).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.3f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_cr))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_cm, columns=labels_cm)
    plt.figure(figsize=(10,10))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    if flag_savefig:
        plt.savefig("confusion_matrix_" + model_ensemble.name + ".png")

    plt.plot()

# 5.Experiment

In [ ]:
if flag_experiment:
    time_start = time.perf_counter()

    print(f'repeats: {repeats}, epochs: {epochs}, batch_size: {batch_size}')
    if flag_EarlyStopping:
        print(f' EarlyStopping is set (patience: {es_patience})\n')
    else:
        print(f' no EarlyStopping\n')

    ## CNN 1D
    run_experiment(modelname_cnn_1d,
                   X_train, X_val, X_test,
                   y_train, y_val, y_test,
                   repeats=repeats)

    ## LSTM Mto1
    run_experiment(modelname_lstm_Mto1,
                   X_train, X_val, X_test,
                   y_train, y_val, y_test,
                   repeats=repeats)

    ## CNN1D-LSTM
    run_experiment(modelname_cnn1d_lstm,
                   X_train_cnn1d_lstm, X_val_cnn1d_lstm, X_test_cnn1d_lstm,
                   y_train, y_val, y_test,
                   repeats=repeats)

    ## ConvLSTM
    run_experiment(modelname_convlstm,
                   X_train_convlstm, X_val_convlstm, X_test_convlstm,
                   y_train, y_val, y_test,
                   repeats=repeats)
    
    print_execution_time(time_start)